## Imports

In [121]:
import pandas as pd
from datetime import datetime
import os
import csv 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Load Data

In [122]:
# Load Dataframe
df_train = pd.read_csv('path-to-file-train.csv')

### Data Preprocessing 

In [3]:
#Introduce lag features
df_train['glucose_lag30'] = df_train['glucose'].shift(6)
df_train['carbs_lag30'] = df_train['carbs'].shift(6)
df_train['bolus_dose_lag30'] = df_train['bolus_dose'].shift(6)

In [216]:
# Data preprocess function
def preprocess_data(df):
    # Convert timestamp to date time 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 

    # Sort by timestamp to maintain proper sequence
    df = df.sort_values('timestamp').reset_index(drop=True)
   
    # Scaling the features using standard scaling
    scaler = StandardScaler()
    df[['glucose', 'basal_rate', 'bolus_dose', 'carbs', 'glucose_lag30', 'carbs_lag30', 'bolus_dose_lag30']] = \
    scaler.fit_transform(df[['glucose', 'basal_rate', 'bolus_dose', 'carbs', 'glucose_lag30', 'carbs_lag30', 'bolus_dose_lag30']])

    return df


In [5]:
# Apply preprocessing
df_train_processed = preprocess_data(df_train)

### Extended Kalman Filter Model Development/Training

In [6]:
# Initialize EKF parameters
x = np.array([30.0])  # Initial ISF estimate, use a value within normal range (30-50)
P = np.array([[0.1]])  # Initial covariance matrix, represents the uncertainty in ISF estimate
Q = np.array([[0.05]])  # Process noise, controls the extent of state change between time steps
R = np.array([[0.01]])  # Measurement noise, controls the noise in glucose measurements

# Prepare feature matrix 'u', for the state transition and measurement models
u = np.zeros((df_train_processed.shape[0], 6))    # Create a zero-filled matrix with 6 columns for the features
u[:, 0] = df_train_processed['glucose']
u[:, 1] = df_train_processed['basal_rate']
u[:, 2] = df_train_processed['bolus_dose']
u[:, 3] = df_train_processed['carbs']
u[:, 4] = df_train_processed['glucose_lag30']
u[:, 5] = df_train_processed['carbs_lag30']   

# Define the nonlinear state transition function
def nonlinear_state_transition(x, u):
    
    # Unpack the input features (u)
    glucose, basal_rate, bolus_dose, carbs, glucose_lag30, carbs_lag30 = u

    # Estimate the effects of bolus insulin, carbohydrates, and basal insulin on ISF
    bolus_effect = 0.001 * (bolus_dose - 0.5)  # Bolus insulin effect on ISF
    carbs_effect = -0.001 * carbs   # Carbohydrate effect 
    basal_effect = 0.001 * basal_rate   # Basal insulin effect
    
    # Predict the next state (ISF) by adding the effects to the current ISF
    x_pred = x + bolus_effect + carbs_effect + basal_effect
    
    return x_pred  # Return the predicted ISF

# Define the nonlinear measurement model to predict glucose levels
def nonlinear_measurement_model(x, u, params):
    
    # Unpack the input features (u)
    glucose, basal_rate, bolus_dose, carbs, glucose_lag30, carbs_lag30 = u
    
     # Unpack additional parameters (e.g., weights for lagged features)
    lag_weight_glucose, lag_weight_carbs = params        
    
     # Estimate the predicted glucose using ISF, insulin (basal + bolus), and carb intake
    z_pred = glucose - (
        x[0] * (0.5 * basal_rate + 0.6 * bolus_dose + 0.4 * carbs) + 
        lag_weight_glucose * glucose_lag30 +  # Influence of lagged glucose
        lag_weight_carbs * carbs_lag30        # Influence of lagged carbs
    )
    return z_pred         # Return the predicted glucose value


# EKF Prediction Step (this predicts the next state and uncertainty)
def predict(x, P, u, Q):
    
    F = np.array([[1]])  # Jacobian of the state transition (identity because we are using linearization)
    
    # Predict the next state (ISF) using the state transition function
    x_pred = nonlinear_state_transition(x, u)

    # Predict the next covariance (uncertainty in the ISF estimate)
    P_pred = np.dot(F, np.dot(P, F.T)) + Q   # Adding process noise (Q) to the covariance
    
    return x_pred, P_pred  # Return the predicted state and covariance

# EKF Update Step (this adjusts the state prediction based on the measurement)
def update(x_pred, P_pred, z, u, R, params):
    
    H = np.array([[1]])  # Jacobian of the measurement model (identity for simplicity)
    
    # Predict the glucose measurement using the measurement model
    z_pred = nonlinear_measurement_model(x_pred, u, params)
    
    # Calculate the residual (difference between actual and predicted glucose)
    y = z - z_pred  # This is the innovation (residual) term
    
     # Calculate the innovation covariance (S) and add measurement noise (R)
    S = np.dot(H, np.dot(P_pred, H.T)) + R
    
     # Regularize S if it's near zero (to avoid numerical instability)
    if np.linalg.det(S) == 0 or np.abs(np.linalg.det(S)) < 1e-6:
        S += 1e-6  # Add a small value to ensure it's invertible
    
    # Kalman Gain calculation: determines how much to adjust the state estimate
    K = np.dot(P_pred, np.dot(H.T, np.linalg.inv(S)))
    
    # Update the state estimate with the Kalman Gain and residual
    x_updated = np.clip(x_pred + np.dot(K, y), 20, 60)  # Keep ISF within reasonable bounds
    
    # Update the covariance matrix (reducing uncertainty after incorporating the measurement)
    P_updated = P_pred - np.dot(K, np.dot(H, P_pred))
    
    return x_updated, P_updated  # Return the updated state and covariance

# Initialize a list to store ISF estimates over time
isf_estimates = []

# Define parameters (weights) for lagged features
params = (0.5, 0.7) 

# Skip the first hour of predictions 
skip_steps = 12  # 12 steps for 5-minute intervals

# Loop through the dataset starting from the time index after the skipped steps
for t in range(skip_steps, len(df_train_processed)):

    # Get the current glucose measurement (z)
    z = np.array([df_train_processed['glucose'].iloc[t]])

    # Predict the next state (ISF) and covariance
    x_pred, P_pred = predict(x, P, u[t, :], Q)

    # Update the state estimate based on the actual glucose measurement
    x, P = update(x_pred, P_pred, z, u[t, :], R, params)
    
    # Handle any overflow or NaN values (if the model produces invalid results)
    if np.any(np.isinf(x)) or np.any(np.isnan(x)):
        print(f"Warning: Overflow or NaN detected at time index {t}")

    # Append the updated ISF estimate to the list
    isf_estimates.append(x[0])
    
# Fill the skipped steps with NaN to align the time index
isf_estimates = [np.nan] * skip_steps + isf_estimates

# Add the ISF predictions to the processed dataframe
df_train_processed['isf_predictions'] = isf_estimates

In [7]:
# Display a sample of the results, skipping the 1st hour
df_train_processed[['timestamp', 'isf_predictions']].iloc[11:26]

# Examine distribution of predictions
df_train_processed['isf_predictions'].describe()

count    12396.000000
mean        39.887879
std         19.478007
min         20.000000
25%         20.000000
50%         35.492992
75%         60.000000
max         60.000000
Name: isf_predictions, dtype: float64

### Model Testing and Evaluation 

In [281]:
# Load Dataframe
df_test = pd.read_csv('path-to-file-test.csv')

# Introduce lag features
df_test['glucose_lag30'] = df_test['glucose'].shift(6)
df_test['carbs_lag30'] = df_test['carbs'].shift(6)
df_test['bolus_dose_lag30'] = df_test['bolus_dose'].shift(6)
df_test = df_test.dropna()

### Calculate ISF using 1700 Rule (ISF = 1700/Total Daily Dose)

In [283]:
# Convert timestamp to datetime if not already
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

# Set timestamp as index
df_test.set_index('timestamp', inplace=True)

In [284]:
# Resample to daily intervals and calculate the sum of basal insulin doses
df_daily_basals = df_test.resample('D').agg({
    'basal_rate': 'mean',  # Average basal rate for the day
    'bolus_dose': 'sum'    # Total bolus dose for the day
}).reset_index()

# Calculate total daily insulin dose (TDD) for each day
df_daily_basals['total_insulin_dose'] = df_daily_basals['basal_rate'] * 24 + df_daily_basals['bolus_dose']  # Assuming basal_rate is in units per hour

# Compute ISF using the 1700 rule
df_daily_basals['isf_1700'] = 1700 / df_daily_basals['total_insulin_dose'].replace(0, np.nan)

### Predict ISF using EKF Model

In [286]:
# Load Dataframe
df_test = pd.read_csv('path-to-file-test.csv')

# Introduce lag features
df_test['glucose_lag30'] = df_test['glucose'].shift(6)
df_test['carbs_lag30'] = df_test['carbs'].shift(6)
df_test['bolus_dose_lag30'] = df_test['bolus_dose'].shift(6)
df_test = df_test.dropna()

In [287]:
# Apply preprocessing and save the scaler
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test_processed = preprocess_data(df_test)

# Initialize EKF parameters
x = np.array([30.0])  
P = np.array([[0.1]]) 
Q = np.array([[0.05]]) 
R = np.array([[0.01]])  

# Adjust the shape of u to include all necessary features
u = np.zeros((df_test_processed.shape[0], 6))  # Now with 6 columns
u[:, 0] = df_test_processed['glucose']
u[:, 1] = df_test_processed['basal_rate']
u[:, 2] = df_test_processed['bolus_dose']
u[:, 3] = df_test_processed['carbs']
u[:, 4] = df_test_processed['glucose_lag30']  # Use 30-minute lag
u[:, 5] = df_test_processed['carbs_lag30']    # Use 30-minute lag

# Initialize a list to store ISF estimates over time
isf_estimates = []

# Define parameters (weights) for lagged features
params = (0.5, 0.7)

# Skip the first hour of predictions 
skip_steps = 12  # 12 steps for 5-minute intervals

# Loop through the dataset starting from the time index after the skipped steps
for t in range(skip_steps, len(df_test_processed)):
    
    # Get the current glucose measurement (z)
    z = np.array([df_test_processed['glucose'].iloc[t]])

    # Predict the next state (ISF) and covariance
    x_pred, P_pred = predict(x, P, u[t, :], Q)

    # Update the state estimate based on the actual glucose measurement
    x, P = update(x_pred, P_pred, z, u[t, :], R, params)
    
    # Handle any overflow or NaN values (if the model produces invalid results)
    if np.any(np.isinf(x)) or np.any(np.isnan(x)):
        print(f"Warning: Overflow or NaN detected in state update at time index {t}")

    # Append the updated ISF estimate to the list
    isf_estimates.append(x[0])

# Fill the skipped steps with NaN to align the time index
isf_estimates = [np.nan] * skip_steps + isf_estimates

# Add the ISF predictions to the processed dataframe
df_test_processed['isf_predictions'] = isf_estimates

##### Aggregate EKF-ISF results into hourly estimates

In [ ]:
# Ensure the 'timestamp' is already in datetime format 
df_test_processed['timestamp'] = pd.to_datetime(df_test_processed['timestamp'])

# Set the timestamp as the DataFrame index (if not already)
df_test_processed.set_index('timestamp', inplace=True)

# Resample to hourly intervals and take the mean of 'isf_predictions' for each hour
df_hourly_isf = df_test_processed['isf_predictions'].resample('H').mean()

# Reset index to get the 'timestamp' back as a column
df_hourly_isf = df_hourly_isf.reset_index()

# Now df_hourly_isf contains hourly timestamp and averaged ISF predictions
#df_hourly_isf.head(15)

### Combine the EKF-ISF and 1700-ISF values into a dataframe

In [ ]:
# Convert the timestamp in df_daily_basals to datetime
df_daily_basals['timestamp'] = pd.to_datetime(df_daily_basals['timestamp'])

# Resample the 'isf_1700' column from df_daily_basals to match the hourly timestamps in df_hourly_isf
# Use the timestamps in df_hourly_isf as the reference for the resampling
df_daily_basals_resampled = df_daily_basals.set_index('timestamp')['isf_1700'].reindex(df_hourly_isf['timestamp'], method='ffill')

# Combine df_hourly_isf with the resampled df_daily_basals
df_hourly_isf['isf_1700'] = df_daily_basals_resampled.values

# Keep only the 'timestamp', 'isf_predictions', and 'isf_1700' columns
df_combined = df_hourly_isf[['timestamp', 'isf_predictions', 'isf_1700']]

# Drop first value
df_combined = df_combined.drop(df_combined.index[0])

# Display the combined DataFrame
df_combined.head(15)

### Evaluate results, by checking expected glucose values 

In [290]:
# Load Dataframe
df_test = pd.read_csv('path-to-file-test.csv')

# Introduce lag features
df_test['glucose_lag30'] = df_test['glucose'].shift(6)
df_test['carbs_lag30'] = df_test['carbs'].shift(6)
df_test['bolus_dose_lag30'] = df_test['bolus_dose'].shift(6)
df_test = df_test.dropna()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define a more complex function to simulate expected glucose levels with delayed insulin effect
def calculate_expected_glucose_after_delay(isf, current_glucose, bolus_dose, carbs, 
                                            delay_factor=0.5, peak_time=60, decay_rate=0.1, 
                                            carb_absorption_delay=30, individual_variability=0.05):
   
    # Simulate insulin effect profile with peak and decay
    time_since_bolus = 30  # Assuming we're evaluating 30 minutes post-bolus
    if time_since_bolus < peak_time:
        insulin_effect = isf * bolus_dose * delay_factor * (time_since_bolus / peak_time)
    else:
        insulin_effect = isf * bolus_dose * delay_factor * ((1 - decay_rate) + decay_rate 
                                                            * (peak_time / time_since_bolus))

    # Simulate delayed carb absorption
    carb_effect = carbs * (1 - np.exp(-time_since_bolus / carb_absorption_delay))
    
    # Apply individual variability
    variability = np.random.uniform(-individual_variability, individual_variability)
    
    # Simulate expected glucose considering the insulin action profile, delayed carb absorption
    expected_glucose = current_glucose - insulin_effect + carb_effect + variability
    
    return expected_glucose

# Ensure 'timestamp' column is in datetime format for both test and combined dataframes
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_combined['timestamp'] = pd.to_datetime(df_combined['timestamp'])

# Set timestamp as index for df_test and resample glucose data to hourly intervals
df_test.set_index('timestamp', inplace=True)
df_test_hourly = df_test.resample('H').mean().reset_index()

# Drop columns that are not needed for aggregation
df_test_hourly.drop(columns=['basal_rate', 'sleep_duration_hours', 'bolus_duration_mins', 
                             'exercise_duration_mins', 'bolus_flag', 'meal_flag', 
                             'exercise_flag', 'hour_of_day'], inplace=True)

# Merge df_combined with df_test_hourly to align the timestamps
df_merged = pd.merge(df_combined, df_test_hourly, on='timestamp')

# Apply the updated function to estimate glucose levels based on ISF predictions
df_merged['expected_glucose_isf'] = df_merged.apply(lambda row: calculate_expected_glucose_after_delay(
    row['isf_predictions'], row['glucose'], row['bolus_dose'], row['carbs']), axis=1)

# Apply the updated function to estimate glucose levels based on the 1700 rule
df_merged['expected_glucose_1700'] = df_merged.apply(lambda row: calculate_expected_glucose_after_delay(
    row['isf_1700'], row['glucose'], row['bolus_dose'], row['carbs']), axis=1)

df_merged = df_merged.dropna()

# Calculate errors for ISF predictions
mae_isf = mean_absolute_error(df_merged['glucose'], df_merged['expected_glucose_isf'])
mse_isf = mean_squared_error(df_merged['glucose'], df_merged['expected_glucose_isf'])
r2_isf = r2_score(df_merged['glucose'], df_merged['expected_glucose_isf'])

# Calculate errors for 1700 rule
mae_1700 = mean_absolute_error(df_merged['glucose'], df_merged['expected_glucose_1700'])
mse_1700 = mean_squared_error(df_merged['glucose'], df_merged['expected_glucose_1700'])
r2_1700 = r2_score(df_merged['glucose'], df_merged['expected_glucose_1700'])

print(f"MAE for ISF Predictions: {mae_isf}")
print(f"MSE for ISF Predictions: {mse_isf}")
print(f"R^2 for ISF Predictions: {r2_isf}")

print(f"MAE for 1700 Rule: {mae_1700}")
print(f"MSE for 1700 Rule: {mse_1700}")
print(f"R^2 for 1700 Rule: {r2_1700}")